<a href="https://colab.research.google.com/github/harelm4/Embible/blob/master/notebooks/mBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 5.4 MB/s 
     |████████████████████████████████| 132 kB 75.8 MB/s 
     |████████████████████████████████| 182 kB 67.2 MB/s 
     |████████████████████████████████| 212 kB 71.0 MB/s 
     |████████████████████████████████| 127 kB 75.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.5 MB/s 
     |████████████████████████████████| 7.6 MB 71.7 MB/s 


In [2]:
from transformers import AutoTokenizer,AutoModelForMaskedLM
cp = "bert-base-multilingual-uncased"
model=AutoModelForMaskedLM.from_pretrained(cp)
tokenizer=AutoTokenizer.from_pretrained(cp)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [3]:
import datasets
ds=datasets.load_dataset("csv", data_files='/content/drive/Shareddrives/Embible/Data/train_df.csv',sep="\t")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-79c7ec62ce633f86/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
def tokenize_function(dataset):
  return tokenizer(str(dataset["verse"])
                      )

tokenized_ds=ds.map(tokenize_function)

  0%|          | 0/22150 [00:00<?, ?ex/s]

In [5]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [21]:
from transformers import TrainingArguments


batch_size = 16
# Show the training loss with every epoch
logging_steps = len(ds["train"])

training_args = TrainingArguments(
    output_dir="/content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mbert",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-3,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    logging_steps=logging_steps,

)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [22]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [23]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [24]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: verse, Unnamed: 0, verse_idx, name. If verse, Unnamed: 0, verse_idx, name are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 22150
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4155
  Number of trainable parameters = 167463831


Epoch,Training Loss,Validation Loss
1,No log,5.728770
2,No log,5.702255
3,No log,5.699402


Saving model checkpoint to /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mbert/checkpoint-500
Configuration saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mbert/checkpoint-500/config.json
Model weights saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mbert/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mbert/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mbert/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mbert/checkpoint-1000
Configuration saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mbert/checkpoint-1000/config.json
Model weights saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mbert

TrainOutput(global_step=4155, training_loss=5.741849522412756, metrics={'train_runtime': 1887.5957, 'train_samples_per_second': 35.204, 'train_steps_per_second': 2.201, 'total_flos': 1923013026013320.0, 'train_loss': 5.741849522412756, 'epoch': 3.0})

In [26]:
trainer.save_model("/content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mBert")
model.save_pretrained("/content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mBert")

Saving model checkpoint to /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mBert
Configuration saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mBert/config.json
Model weights saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mBert/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mBert/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mBert/special_tokens_map.json
Configuration saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mBert/config.json
Model weights saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-mBert/pytorch_model.bin
